In [ ]:
!curl -L -o /content/Fashion_MNIST/fashionmnist.zip https://www.kaggle.com/api/v1/datasets/download/zalando-research/fashionmnist
!unzip /content/Fashion_MNIST/fashionmnist.zip -d /content/Fashion_MNIST/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 68.7M  100 68.7M    0     0  11.8M      0  0:00:05  0:00:05 --:--:-- 16.4M
Archive:  /content/Fashion_MNIST/fashionmnist.zip
  inflating: /content/Fashion_MNIST/fashion-mnist_test.csv  
  inflating: /content/Fashion_MNIST/fashion-mnist_train.csv  
  inflating: /content/Fashion_MNIST/t10k-images-idx3-ubyte  
  inflating: /content/Fashion_MNIST/t10k-labels-idx1-ubyte  
  inflating: /content/Fashion_MNIST/train-images-idx3-ubyte  
  inflating: /content/Fashion_MNIST/train-labels-idx1-ubyte  


In [ ]:
!pip install torchinfo
import torch, torchinfo


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('/content/Fashion_MNIST/fashion-mnist_train.csv')
X = df.iloc[:,1:].values
y = df.iloc[:,0].values

X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.25, random_state=69)
X_train = X_train/255.0
x_test = X_test/255.0


In [ ]:
from torch.utils.data import Dataset, DataLoader
class CustomDataset(torch.utils.data.Dataset):
  def __init__(self, features, labels):
    self.features = torch.tensor(features , dtype = torch.float32)
    self.labels = torch.tensor(labels, dtype=torch.long)
  def __len__(self):
    return len(self.features)
  def __getitem__(self, index):
    return self.features[index], self.labels[index]



In [ ]:
train_dataset = CustomDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory= True)
test_dataset = CustomDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory= True)

In [ ]:
import torch.nn as nn

class NN(torch.nn.Module):
  def __init__(self, num_features):
    super().__init__()
    self.model = nn.Sequential(
        nn.Linear(num_features, 128),
        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.Dropout(p=0.25),
        nn.Linear(128, 64),
        nn.BatchNorm1d(64),
        nn.ReLU(),
        nn.Dropout(p=0.25),
        nn.Linear(64, 10)
    )

  def forward(self, x):
    return self.model(x)

epochs = 100
learning_rate = 0.1

In [ ]:
model = NN(X_train.shape[1])

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate,weight_decay = 1e-4)


In [ ]:
for epoch in range(epochs):
  total_epochs_loss = 0
  for batch_idx, (data, targets) in enumerate(train_loader):
    preds = model(data)

    loss = criterion(preds,targets)
    optimizer.zero_grad()
    loss.backward()

    optimizer.step()
    total_epochs_loss = total_epochs_loss + loss.item()

  avg_loss = total_epochs_loss/len(train_loader)
  print(f'Epoch: {epoch+1}, Loss: {avg_loss}')


Epoch: 1, Loss: 0.6091243498657474
Epoch: 2, Loss: 0.4737097313197352
Epoch: 3, Loss: 0.4396821263205861
Epoch: 4, Loss: 0.4158070398694501
Epoch: 5, Loss: 0.3951145104783261
Epoch: 6, Loss: 0.3832135838062609


KeyboardInterrupt: 

In [ ]:
model.eval()
total = 0
correct = 0

with torch.no_grad():
  for data, targets in test_loader:
    preds = model(data)

    _, predicted = torch.max(preds.data, 1)
    total += targets.size(0)
    correct += (predicted == targets).sum().item()

print(f'Accuracy: {100*correct/total}')

# Accuracy Train VS Test set

total_2 = 0
correct_2 = 0

with torch.no_grad():
  for data, targets in train_loader:
    preds = model(data)

    _, predicted = torch.max(preds.data, 1)
    total_2 += targets.size(0)
    correct_2 += (predicted == targets).sum().item()
print(f'Accuracy: {100*correct_2/total_2}')

# Test accuracy decreased by 1% and on Train is by 9% which helps in avoiding overfitting


Accuracy: 80.08
Accuracy: 89.54222222222222


In [ ]:
import torch.nn as nn

class NN(torch.nn.Module):
  def __init__(self, num_features):
    super().__init__()
    self.model = nn.Sequential(
        nn.Linear(num_features, 128),
        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.Dropout(p=0.25),
        nn.Linear(128, 64),
        nn.BatchNorm1d(64),
        nn.ReLU(),
        nn.Dropout(p=0.25),
        nn.Linear(64, 10)
    )

  def forward(self, x):
    return self.model(x)

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
import torch.nn as nn

class MyNN(nn.Module):
  def __init__(self,input_dims,output_dim, num_hidden_layers, neurons_per_layer):
    super().__init__()

    layers = []

    for i in range(num_hidden_layers):
      layers.append(nn.Linear(input_dims,neurons_per_layer))
      layers.append(nn.BatchNorm1d(neurons_per_layer))
      layers.append(nn.ReLU())
      layers.append(nn.Dropout(p=0.25))
      input_dims = neurons_per_layer

    layers.append(nn.Linear(input_dims,output_dim))

    self.model = nn.Sequential(*layers)

  def forward(self,x):
    return self.model(x)


#objective function
def objective(trial):

  #State Search
  num_hidden_layers =   trial.suggest_int("num_hidden_layers" ,1,5)
  neurons_per_layer = trial.suggest_int("neurons_per_layer" ,8,128,step = 8)

  #model init
  input_dim = 784
  output_dim = 10
  model = MyNN(input_dim,output_dim, num_hidden_layers, neurons_per_layer)
  model.to(device)  # Move model to device

  #loss function
  criterion = nn.CrossEntropyLoss()

  #params init
  learning_rate = 0.01
  epochs = 50

  #optimizer selection
  optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate,weight_decay = 1e-4)


  #training loop
  for epoch in range(epochs):
    for data, targets in train_loader:
      data, targets = data.to(device), targets.to(device) # Move data to device
      preds = model(data)

      loss = criterion(preds,targets)
      optimizer.zero_grad()
      loss.backward()

      optimizer.step()
  #evaluation
  model.eval()
  total = 0
  correct = 0

  with torch.no_grad():
    for data, targets in test_loader:
      data, targets = data.to(device), targets.to(device) # Move data to device
      preds = model(data)

      _, predicted = torch.max(preds.data, 1)
      total += targets.size(0)
      correct += (predicted == targets).sum().item()
      accuracy = 100*correct/total

  return accuracy

In [ ]:
!pip install optuna

In [ ]:
import optuna

study = optuna.create_study(direction = 'maximize')

[I 2025-02-05 10:51:37,017] A new study created in memory with name: no-name-b6aa8ce6-46f3-411f-b119-b7e4f5f7d646


In [ ]:
study.optimize(objective, n_trials=10)

[I 2025-02-05 10:55:33,478] Trial 0 finished with value: 81.35333333333334 and parameters: {'num_hidden_layers': 4, 'neurons_per_layer': 112}. Best is trial 0 with value: 81.35333333333334.
[I 2025-02-05 10:59:47,786] Trial 1 finished with value: 29.026666666666667 and parameters: {'num_hidden_layers': 5, 'neurons_per_layer': 8}. Best is trial 0 with value: 81.35333333333334.
[I 2025-02-05 11:02:25,916] Trial 2 finished with value: 81.16 and parameters: {'num_hidden_layers': 2, 'neurons_per_layer': 72}. Best is trial 0 with value: 81.35333333333334.
[I 2025-02-05 11:05:05,019] Trial 3 finished with value: 82.02 and parameters: {'num_hidden_layers': 2, 'neurons_per_layer': 96}. Best is trial 3 with value: 82.02.
[I 2025-02-05 11:08:15,931] Trial 4 finished with value: 82.13333333333334 and parameters: {'num_hidden_layers': 3, 'neurons_per_layer': 48}. Best is trial 4 with value: 82.13333333333334.
[I 2025-02-05 11:10:55,580] Trial 5 finished with value: 83.79333333333334 and parameters:

In [ ]:
print(study.best_params)
print(study.best_trial)

{'num_hidden_layers': 2, 'neurons_per_layer': 80}
FrozenTrial(number=5, state=1, values=[83.79333333333334], datetime_start=datetime.datetime(2025, 2, 5, 11, 8, 15, 937463), datetime_complete=datetime.datetime(2025, 2, 5, 11, 10, 55, 580414), params={'num_hidden_layers': 2, 'neurons_per_layer': 80}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'num_hidden_layers': IntDistribution(high=5, log=False, low=1, step=1), 'neurons_per_layer': IntDistribution(high=128, log=False, low=8, step=8)}, trial_id=5, value=None)


In [ ]:
import torch.nn as nn

class MyNN(nn.Module):
  def __init__(self,input_dims,output_dim, num_hidden_layers, neurons_per_layer,dropout_rate):
    super().__init__()

    layers = []

    for i in range(num_hidden_layers):
      layers.append(nn.Linear(input_dims,neurons_per_layer))
      layers.append(nn.BatchNorm1d(neurons_per_layer))
      layers.append(nn.ReLU())
      layers.append(nn.Dropout(dropout_rate))
      input_dims = neurons_per_layer

    layers.append(nn.Linear(input_dims,output_dim))

    self.model = nn.Sequential(*layers)

  def forward(self,x):
    return self.model(x)


#objective function
def objective(trial):

  #State Search
  num_hidden_layers =   trial.suggest_int("num_hidden_layers" ,1,5)
  neurons_per_layer = trial.suggest_int("neurons_per_layer" ,8,128,step = 8)
  epochs = trial.suggest_int("epochs" , 10, 100, step = 10)
  learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
  dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5, step=0.1)
  batch_size = trial.suggest_categorical("batch_size", [16,32,64,128])
  optimizer_name = trial.suggest_categorical("optimizer_name" , ['Adam', 'SGD', 'RMSprop'])
  weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-1, log=True)

  #model init
  input_dim = 784
  output_dim = 10
  model = MyNN(input_dim,output_dim, num_hidden_layers, neurons_per_layer,dropout_rate)
  model.to(device)  # Move model to device


  train_dataset = CustomDataset(X_train, y_train)
  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory= True)
  test_dataset = CustomDataset(X_test, y_test)
  test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory= True)

  #loss function
  criterion = nn.CrossEntropyLoss()

  #params init

  #optimizer selection
  if optimizer_name == 'Adam':
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay = weight_decay)
  elif optimizer_name == 'SGD':
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate,weight_decay = weight_decay)
  else:
    optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate,weight_decay = weight_decay)

  #training loop
  for epoch in range(epochs):
    for data, targets in train_loader:
      data, targets = data.to(device), targets.to(device) # Move data to device
      preds = model(data)

      loss = criterion(preds,targets)
      optimizer.zero_grad()
      loss.backward()

      optimizer.step()
  #evaluation
  model.eval()
  total = 0
  correct = 0

  with torch.no_grad():
    for data, targets in test_loader:
      data, targets = data.to(device), targets.to(device) # Move data to device
      preds = model(data)

      _, predicted = torch.max(preds.data, 1)
      total += targets.size(0)
      correct += (predicted == targets).sum().item()
      accuracy = 100*correct/total

  return accuracy

In [ ]:
import optuna

study = optuna.create_study(direction = 'maximize')
study.optimize(objective, n_trials=10)

[I 2025-02-05 11:37:48,121] A new study created in memory with name: no-name-0e1c545b-8b84-447f-b2b3-a6221fa9431b
[I 2025-02-05 11:40:45,628] Trial 0 finished with value: 65.34 and parameters: {'num_hidden_layers': 2, 'neurons_per_layer': 104, 'epochs': 90, 'learning_rate': 0.007928745622192982, 'dropout_rate': 0.5, 'batch_size': 64, 'optimizer_name': 'Adam', 'weight_decay': 0.0029908089900752555}. Best is trial 0 with value: 65.34.
[I 2025-02-05 11:43:05,058] Trial 1 finished with value: 70.74 and parameters: {'num_hidden_layers': 1, 'neurons_per_layer': 80, 'epochs': 90, 'learning_rate': 0.0010284710071389823, 'dropout_rate': 0.1, 'batch_size': 64, 'optimizer_name': 'RMSprop', 'weight_decay': 0.0022222267682185082}. Best is trial 1 with value: 70.74.
[I 2025-02-05 11:44:18,353] Trial 2 finished with value: 72.68 and parameters: {'num_hidden_layers': 3, 'neurons_per_layer': 80, 'epochs': 60, 'learning_rate': 0.017194953417692668, 'dropout_rate': 0.1, 'batch_size': 128, 'optimizer_name

In [ ]:
study.optimize(objective, n_trials=10)

[I 2025-02-05 11:54:56,216] Trial 10 finished with value: 48.06666666666667 and parameters: {'num_hidden_layers': 2, 'neurons_per_layer': 96, 'epochs': 10, 'learning_rate': 1.0253643833231317e-05, 'dropout_rate': 0.2, 'batch_size': 16, 'optimizer_name': 'SGD', 'weight_decay': 0.08510751785539593}. Best is trial 7 with value: 80.91333333333333.
[I 2025-02-05 11:57:28,602] Trial 11 finished with value: 80.07333333333334 and parameters: {'num_hidden_layers': 1, 'neurons_per_layer': 48, 'epochs': 30, 'learning_rate': 2.9714068613000976e-05, 'dropout_rate': 0.4, 'batch_size': 16, 'optimizer_name': 'RMSprop', 'weight_decay': 0.00011914776857871813}. Best is trial 7 with value: 80.91333333333333.
[I 2025-02-05 11:59:10,508] Trial 12 finished with value: 76.79333333333334 and parameters: {'num_hidden_layers': 1, 'neurons_per_layer': 56, 'epochs': 20, 'learning_rate': 1.5668232083875016e-05, 'dropout_rate': 0.4, 'batch_size': 16, 'optimizer_name': 'RMSprop', 'weight_decay': 8.675793351114669e-0